#DEMO NOTEBOOK FOR PUI2015
##__author__ FBB

this is to dempnstrate the power of vectorization: lets create a table with the cells for the chisq test via a nested for loop (of only 2 elements!) and with vectorized operations

In [1]:
import pylab as pl
import pandas as pd
import numpy as np
%pylab inline

import os

import json
s = json.load( open(os.getenv ('PUI2015')+"/fbb_matplotlibrc.json") )
pl.rcParams.update(s)


Populating the interactive namespace from numpy and matplotlib


/Users/fbianco/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:855: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


collecting the data

In [31]:
df=pd.read_csv(os.getenv ('PUI2015')+'/notebooks/data/201501-citibike-tripdata.csv')
print df.columns

#df is the dataframe where the content of the csv file is stored

df['ageM'] = 2015-df['birth year'][(df['usertype']=='Subscriber') & (df['gender']==1)]
df['ageF'] = 2015-df['birth year'][(df['usertype']=='Subscriber') & (df['gender']==2)]


bins = np.arange(0, 99, 10)

cs1=df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).cumsum()

cs2=df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).cumsum()


Index([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender'],
      dtype='object')


creating the bin counts the data

In [32]:
values=[np.empty_like(bins), np.empty_like(bins)]
values[0]=np.array(cs1/cs1.max())
values[1]=np.array(cs2/cs2.max())
#here i was testing
#values=np.array([[1,2,3],[1,2,3]])
print values
values=np.array(values).squeeze(axis=2)
print values
values[np.isnan(values)]=0

[array([[        nan],
       [ 0.0083646 ],
       [ 0.23188476],
       [ 0.54738035],
       [ 0.79106956],
       [ 0.94736102],
       [ 0.99366505],
       [ 0.99977156],
       [ 1.        ]]), array([[        nan],
       [ 0.01002054],
       [ 0.29282245],
       [ 0.59948554],
       [ 0.80966733],
       [ 0.95824775],
       [ 0.99679419],
       [ 0.99988482],
       [ 1.        ]])]
[[        nan  0.0083646   0.23188476  0.54738035  0.79106956  0.94736102
   0.99366505  0.99977156  1.        ]
 [        nan  0.01002054  0.29282245  0.59948554  0.80966733  0.95824775
   0.99679419  0.99988482  1.        ]]


creating the cells with c-style loops

In [33]:
%%timeit -n 100
E = np.empty_like(values)

for j in range(len(values[0])):
    for i in range(2):
        E[i][j] = sum(values[i,:])*sum(values[:,j])/sum(values)
#%whos

100 loops, best of 3: 241 µs per loop


In [34]:
print E 
print len(values[0])
print "chisq" , nansum((values-E)**2/E)
print values-E

[[ 0.          0.00907142  0.25889605  0.56587568  0.78982039  0.94024738
   0.9821135   0.98665146  0.98682101]
 [ 0.          0.00931372  0.26581116  0.58099021  0.81091649  0.96536139
   1.00834574  1.01300492  1.01317899]]
9
chisq 0.00793517321213
[[ 0.         -0.00070682 -0.02701129 -0.01849532  0.00124917  0.00711364
   0.01155155  0.01312009  0.01317899]
 [ 0.          0.00070682  0.02701129  0.01849532 -0.00124917 -0.00711364
  -0.01155155 -0.01312009 -0.01317899]]


creating the cells via vectorized operations

In [35]:
%%timeit -n 100 


E_vec = np.outer(sum(values,axis=1),sum(values,axis=0))
E_vec=(E_vec/sum(values))
#%whos


100 loops, best of 3: 20.3 µs per loop


In [36]:
print E
print len(values[0])
print "chisq" , nansum((values-E)**2/E)
print values-E

[[ 0.          0.00907142  0.25889605  0.56587568  0.78982039  0.94024738
   0.9821135   0.98665146  0.98682101]
 [ 0.          0.00931372  0.26581116  0.58099021  0.81091649  0.96536139
   1.00834574  1.01300492  1.01317899]]
9
chisq 0.00793517321213
[[ 0.         -0.00070682 -0.02701129 -0.01849532  0.00124917  0.00711364
   0.01155155  0.01312009  0.01317899]
 [ 0.          0.00070682  0.02701129  0.01849532 -0.00124917 -0.00711364
  -0.01155155 -0.01312009 -0.01317899]]


## 10 times faster!!